In [ ]:
import tensorflow as tf

In [ ]:
! pip freeze | grep cuda

cupy-cuda11x==11.0.0
jaxlib @ https://storage.googleapis.com/jax-releases/cuda11/jaxlib-0.3.25+cuda11.cudnn805-cp37-cp37m-manylinux2014_x86_64.whl


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
path_default = '/content/drive/MyDrive/2022-2/TCC2/DESENVOLVIMENTO/plate_detection'

# Setup Paths

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet_6'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
DATASETS_URL = 'https://www.kaggle.com/andrewmvd/car-plate-detection/download'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.txt'

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join(path_default, 'Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join(path_default, 'Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join(path_default, 'Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join(path_default, 'Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join(path_default, 'Tensorflow', 'workspace','images'),
    'IMAGE_PATH_TRAIN': os.path.join(path_default, 'Tensorflow', 'workspace','images','train'),
    'IMAGE_PATH_TEST': os.path.join(path_default, 'Tensorflow', 'workspace','images','test'),
    'IMAGE_PATH_EVAL': os.path.join(path_default, 'Tensorflow', 'workspace','images','eval'),
    'IMAGE_PATH_TFLITE': os.path.join(path_default, 'Tensorflow', 'workspace','images','tflite_image'),
    'MODEL_PATH': os.path.join(path_default, 'Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join(path_default, 'Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join(path_default, 'Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME,'checkpoint'), 
    'OUTPUT_PATH': os.path.join(path_default, 'Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFLITE_PATH':os.path.join(path_default, 'Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join(path_default, 'Tensorflow','protoc')
    
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join(path_default, 'Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME,'checkpoint','pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(path_default, paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(path_default, paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        !mkdir -p {path}

# Download Modelo pré-treinado do Tensorflow Model Zoo e API Object detection

In [ ]:
# Baixar modelo, somente na primeira execução
if False:
  !wget {PRETRAINED_MODEL_URL}
  !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
  !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models.git {paths['APIMODEL_PATH']}

In [ ]:
# Install Tensorflow Object Detection 

# !sudo apt-get install protobuf-compiler
!cd {os.path.join(path_default, 'Tensorflow/models/research')} && \
  protoc object_detection/protos/*.proto --python_out=. && \
  cp object_detection/packages/tf2/setup.py . && \
  python -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/drive/MyDrive/2022-2/TCC2/DESENVOLVIMENTO/plate_detection/Tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 11.0 MB 8.0 MB/s 
     |████████████████████████████████| 352 kB 68.8 MB/s 
     |████████████████████████████████| 665.5 MB 24 kB/s 
     |████████████████████████████████| 1.8 MB 48.3 MB/s 
     |████████████████████████████████| 23.1 MB 1.1 MB/s 
     |████████████████████████████████| 67 kB 7.1 MB/s 
     |████████████████████████████████| 463 kB 73.0 MB

In [ ]:
# Verify Installation Object Detection
import object_detection

# Instalando tensorflow, CUDA and cuDNN

In [ ]:
# !sudo apt-get install cuda-11.0

In [ ]:
# !pip install tensorflow==2.5 tensorflow-gpu==2.5 --upgrade

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

# Conjunto de dados, mapeamento de rótulos  e TF records

In [ ]:
print("Fazer o download do link " + DATASETS_URL)

print("\nExtrair as imagens e rotulos, dividindo-os em 95% para treino e 5% teste, movendo-as para seus caminhos:\n")
print(os.path.join(paths['IMAGE_PATH_TRAIN']))
print(os.path.join(paths['IMAGE_PATH_TEST']))

Fazer o download do link https://www.kaggle.com/andrewmvd/car-plate-detection/download

Extrair as imagens e rotulos, dividindo-os em 95% para treino e 5% teste, movendo-as para seus caminhos:

/content/drive/MyDrive/2022-2/TCC2/DESENVOLVIMENTO/plate_detection/Tensorflow/workspace/images/train
/content/drive/MyDrive/2022-2/TCC2/DESENVOLVIMENTO/plate_detection/Tensorflow/workspace/images/test


In [ ]:
files['LABELMAP']

'/content/drive/MyDrive/2022-2/TCC2/DESENVOLVIMENTO/plate_detection/Tensorflow/workspace/annotations/label_map.txt'

In [ ]:
labels = [{'name':'licence', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [ ]:
files['TF_RECORD_SCRIPT']

'/content/drive/MyDrive/2022-2/TCC2/DESENVOLVIMENTO/plate_detection/Tensorflow/scripts/generate_tfrecord.py'

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {paths['IMAGE_PATH_TRAIN']} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {paths['IMAGE_PATH_TEST']} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

# Copiando configurações do modelo pré-treinado e atualizando o pipeline para transferencia de aprendizado

In [ ]:
import tensorflow as tf

In [ ]:
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.height = 320
pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.width = 320
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]


config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

# Treinamento do modelo

In [ ]:
#solução de problemas para o treinamento
# !pip uninstall pycocotools -y
# !pip install pycocotools==2.0.1

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=10000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [ ]:
!{command}

# Avaliação do modelo

In [ ]:
#% Exemplo
#python model_main_tf2.py -- \
#  --model_dir=$MODEL_DIR 
#  --num_train_steps=$NUM_TRAIN_STEPS \
#  --sample_1_of_n_eval_examples=$SAMPLE_1_OF_N_EVAL_EXAMPLES \
#  --pipeline_config_path=$PIPELINE_CONFIG_PATH \
#  --alsologtostderr

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [ ]:
print(command)

python /content/drive/MyDrive/2022-2/TCC2/DESENVOLVIMENTO/plate_detection/Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=/content/drive/MyDrive/2022-2/TCC2/DESENVOLVIMENTO/plate_detection/Tensorflow/workspace/models/my_ssd_mobnet_6/checkpoint --pipeline_config_path=/content/drive/MyDrive/2022-2/TCC2/DESENVOLVIMENTO/plate_detection/Tensorflow/workspace/models/my_ssd_mobnet_6/checkpoint/pipeline.config --checkpoint_dir=/content/drive/MyDrive/2022-2/TCC2/DESENVOLVIMENTO/plate_detection/Tensorflow/workspace/models/my_ssd_mobnet_6/checkpoint


In [ ]:
!{command}

# Congelando modelo e exportando para tflite

In [ ]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])
!{command}

In [ ]:
TFLITE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py ')
command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(TFLITE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['TFLITE_PATH'])
!{command}

In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input
import tensorflow as tf
import numpy as np


def representative_dataset():
  dataset_list = tf.data.Dataset.list_files(os.path.join(paths['IMAGE_PATH_TFLITE'], 'plate/*'))
  for i in range(len(dataset_list)):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [320, 320])
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]


def write_model(converter, save_model, quantization):
    tf_lite_model = converter.convert()
    open(os.path.join(save_model, f'{quantization}.tflite'), 'wb').write(tf_lite_model)


def convert_tflite(quantization, saved_model_dir, allow_ops_tf=False):
    converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

    if quantization == 'original':
        write_model(converter, saved_model_dir, quantization)
    else:
        
        converter.optimizations = [tf.lite.Optimize.DEFAULT]

        # allow operations compatible with tensorflow
        if allow_ops_tf:
            converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
            tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
            ]

        converter.output_arrays = [
            'TFLite_Detection_PostProcess:0',
            'TFLite_Detection_PostProcess:1',
            'TFLite_Detection_PostProcess:2',
            'TFLite_Detection_PostProcess:3'
        ]

        if quantization == 'float16':
            converter.target_spec.supported_types = [tf.float16]
        elif quantization == 'int8' or quantization == 'full_int8':
            converter.input_shapes = 1,320,320,3
            converter.representative_dataset = representative_dataset
        if quantization == 'full_int8':
            converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
            converter.inference_input_type = tf.int8  # or tf.uint8
            converter.inference_output_type = tf.int8  # or tf.uint8

        write_model(converter, save_model, quantization)

In [ ]:
FROZEN_TFLITE_PATH = os.path.join(paths['TFLITE_PATH'], 'saved_model')
TFLITE_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model')

In [ ]:
convert_tflite('original', FROZEN_TFLITE_PATH)

In [ ]:
convert_tflite('dynamic_range_quantization', FROZEN_TFLITE_PATH)

In [ ]:
convert_tflite('float16', FROZEN_TFLITE_PATH)

In [ ]:
convert_tflite('int8', FROZEN_TFLITE_PATH)

Found 433 images belonging to 1 classes.


In [ ]:
convert_tflite('full_int8', FROZEN_TFLITE_PATH)

Found 433 images belonging to 1 classes.


In [ ]:
# model = tf.saved_model.load(os.path.join(paths['TFLITE_PATH'], 'saved_model'))
# assinaturas = model.signatures['serving_default']
# # assinaturas.structured_outputs

# Exemplo de inferência

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)